In [1]:
# IMPORTS
import numpy as np
import hpsklearn
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline 
import matplotlib.pyplot as plt

from hpsklearn import HyperoptEstimator
from hpsklearn import any_classifier
from hpsklearn import any_preprocessing
from hyperopt import tpe
import os
os.environ['OMP_NUM_THREADS'] = "1"

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


In [2]:
Games_dados = pd.read_csv("../dados/GamesDados_v2.csv")

In [3]:
Games_dados['Vendas'] = Games_dados.Global_Sales_GenMed.map(lambda x: x > 0.999999 and 'Alto' or 'Baixo')

In [4]:
variaveis_predicao = ['Code_Genre','Code_Platform','Global_Sales_GenMed']

target = Games_dados['Vendas']
X = Games_dados[variaveis_predicao].to_numpy()
y = target.to_numpy()

In [5]:
# Separação dos Dados teste,treinamento
from sklearn.model_selection import train_test_split

# Hiperparametros
# test_size = 0.3 (30% dos dados para teste)
test_size = 0.3
# ramdom_state = Controls the shuffling applied to the data before applying the split
random_state = 2021

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y,test_size=test_size ,random_state=random_state)

In [6]:
model = HyperoptEstimator(classifier=any_classifier('cla'), 
                          preprocessing=any_preprocessing('pre'), 
                          algo=tpe.suggest, 
                          max_evals=50, 
                          trial_timeout=30)

model.fit(Xtrain,ytrain)

100%|███████████████████████████████████████████████████████████████| 50/50 [00:06<00:00,  6.16s/trial, best loss: 0.0]


In [10]:
# Make a prediction using the optimized model
prediction = model.predict(Xtest)

In [11]:
prediction

array(['Baixo', 'Baixo', 'Baixo', ..., 'Baixo', 'Baixo', 'Baixo'],
      dtype=object)

In [8]:
acc = model.score(Xtest,ytest)
print(acc)

1.0


In [9]:
print(model.best_model())

{'learner': RandomForestClassifier(bootstrap=False, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='log2', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=18, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=352, n_jobs=1,
                       oob_score=False, random_state=1, verbose=False,
                       warm_start=False), 'preprocs': (StandardScaler(copy=True, with_mean=True, with_std=True),), 'ex_preprocs': ()}
